# Projet : Analyse de l'Insertion des Diplômés de Master

1. Présentation du Dataset Dans le cadre du module AAS, nous analysons le jeu de données "Insertion professionnelle des diplômés de Master" (source : data.gouv.fr). Ce dataset recense les indicateurs d'emploi (taux d'insertion, salaires) des diplômés universitaires en France, agrégés par établissement, discipline et année.

2. Objectif et Tâche ML L'objectif est d'identifier les facteurs déterminants de l'employabilité des jeunes diplômés.

Nous formulons ce problème comme une tâche de Régression :

Variable Cible (Target) : taux_dinsertion (le pourcentage de diplômés ayant un emploi).

Démarche : Le projet suivra un pipeline complet : nettoyage des données, exploration visuelle (EDA) et modélisation prédictive.

# 1.Nettoyage

## 1.1 importations du dataset

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('fr-esr-insertion_professionnelle-master.csv', sep=';')
df.head()

,annee,diplome,numero_de_l_etablissement,etablissement,etablissementactuel,code_de_l_academie,academie,code_du_domaine,domaine,code_de_la_discipline,...,taux_de_chomage_regional,salaire_net_mensuel_median_regional,emplois_cadre,emplois_exterieurs_a_la_region_de_luniversite,femmes,salaire_net_mensuel_regional_1er_quartile,salaire_net_mensuel_regional_3eme_quartile,cle_etab,cle_disc,id_paysage
0,2012,MASTER LMD,0681166Y,Mulhouse - Haute Alsace,NaN,A15,Strasbourg,STS,"Sciences, technologies et santé",disc15,...,9.3,1760,ns,ns,ns,1430,2080,Mulhouse - Haute Alsace_18,disc15_18,OJZ4a
1,2012,MASTER LMD,0691775E,Lyon 2 - Lumière,NaN,A10,Lyon,SHS,Sciences humaines et sociales,disc09,...,8.9,1780,81,29,83,1430,2130,Lyon 2 - Lumière_18,disc09_18,CUBKB
2,2012,MASTER LMD,0692437Z,Lyon 3 - Jean Moulin,NaN,A10,Lyon,SHS,Sciences humaines et sociales,disc11,...,8.9,1780,45,41,57,1430,2130,Lyon 3 - Jean Moulin_18,disc11_18,7Gzub
3,2012,MASTER LMD,0730858L,Savoie Mont Blanc,NaN,A08,Grenoble,DEG,"Droit, économie et gestion",disc01,...,8.9,1780,30,46,66,1430,2130,Savoie Mont Blanc_18,disc01_18,zCa4j
4,2012,MASTER LMD,0750736T,Paris - Dauphine,NaN,A01,Paris,DEG,"Droit, économie et gestion",disc04,...,9,2020,NaN,NaN,NaN,1580,2530,Paris - Dauphine_18,disc04_18,C6Ps7


## 1.2 SUPPRESSION DES COLONNES VIDES
On supprime les colonnes qui ont plus de 50% de vide (comme 'remarque' ou 'etablissementactuel')

In [2]:
limit = len(df) * 0.5
df_clean = df.dropna(thresh=limit, axis=1).copy()

## 1.3 CONVERSION DES CHIFFRES
Par sécurité, on force la conversion des colonnes importantes en numérique


In [3]:
cols_to_fix = ['taux_dinsertion', 'salaire_net_median_des_emplois_a_temps_plein']
for col in cols_to_fix:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

## 1.4 NETTOYAGE DES LIGNES
On supprime les lignes où l'info cruciale 'taux_dinsertion' est manquante

In [4]:
target_col = 'taux_dinsertion'
df_clean = df_clean.dropna(subset=[target_col])

## 1.5 IMPUTATION DES VARIABLES EXPLICATIVES (FEATURES)
 Pour ne pas perdre d'info, on remplit les trous des autres colonnes numériques (ex: salaire) par la médiane

In [5]:
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col != target_col: # On ne touche pas à la target car on l'a déjà nettoyée
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

## 1.6 Resultat

In [ ]:
print(f"Lignes avant : {len(df)} -> Lignes après nettoyage : {len(df_clean)}")
df_clean.head()


Lignes avant : 19603 -> Lignes après nettoyage : 10856


PermissionError: [Errno 13] Permission denied: 'mon_fichier_corrigé.xlsx'

# 2. EDA